# Argo data demo

Student: Gerardo Rivera Tello

OCN 620, Fall 2024

In [143]:
from argopy import DataFetcher, IndexFetcher
from argopy.plot import scatter_map
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import gsw
import cmocean as cmo

plt.rcParams["font.family"] = "monospace"

First we need to define the region we want to use to select the available floats

In [ ]:
region = IndexFetcher().region([-161, -154, 18, 23, "2024-01", "2024-10"]).load()
region

We can plot the trajectories with the platform number (wmo code) as the legend

In [ ]:
region.plot("trajectory")

Argopy has nice plotting utilities so we can explore the statistics of the floats

In [ ]:
region.plot("dac")

In [ ]:
region.plot("profiler")

After exploring and playing around with the indexed data, we can switch to the `DataFetcher` which will be used to retrieve the data from the floats

In [9]:
float_data = (
    DataFetcher(mode="expert")
    .region([-161, -154, 18, 23, 0, 2000, "2024-01", "2024-10"])
    .load()
)
ds = float_data.data.argo.point2profile()
df = float_data.index

Again, `argopy` has nice features to do some plotting with different hues that are defined by the data

In [ ]:
scatter_map(ds, hue="DATA_MODE")

We can explore the data by inspecting the `ds` variable

In [ ]:
single_profile = ds.isel(N_PROF=10)
single_profile

In [27]:
Z = gsw.z_from_p(single_profile.PRES.values, single_profile.LATITUDE.values)
SA = gsw.SA_from_SP(
    single_profile.PSAL.values,
    single_profile.PRES.values,
    single_profile.LONGITUDE.values,
    single_profile.LATITUDE.values,
)
CT = gsw.CT_from_t(SA, single_profile.TEMP.values, single_profile.PRES.values)

RHO = gsw.rho(SA, CT, single_profile.PRES.values)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(10, 5), sharey=True)
axs[0].plot(RHO, Z, c="b")
axs[0].set_xlabel("Density (kg/m^3)")
axs[0].set_ylabel("Depth (m)")

axs[1].plot(SA, Z, c="g")
axs[1].set_xlabel("Absolute Salinity (g/kg)")

axs[2].plot(CT, Z, c="r")
axs[2].set_xlabel("Conservative Temperature (C)")

for _ax in axs:
    _ax.grid(ls="--")

We can extend this by doing a hovmoller of these variables for a single float

In [ ]:
WMO = 1902685
single_profile = ds.where(ds.PLATFORM_NUMBER == WMO, drop=True)
single_profile

In [ ]:
scatter_map(single_profile, hue="DATA_MODE")

We do the same computation as before

In [115]:
Z = gsw.z_from_p(
    single_profile.PRES.values,
    np.stack([single_profile.LATITUDE.values] * single_profile.PRES.shape[1], axis=1),
)
SA = gsw.SA_from_SP(
    single_profile.PSAL.values,
    single_profile.PRES.values,
    np.stack([single_profile.LONGITUDE.values] * single_profile.PRES.shape[1], axis=1),
    np.stack([single_profile.LATITUDE.values] * single_profile.PRES.shape[1], axis=1),
)
CT = gsw.CT_from_t(SA, single_profile.TEMP.values, single_profile.PRES.values)

RHO = gsw.rho(SA, CT, single_profile.PRES.values)

Lastly, we convert the variables into xarray dataarrays so we can plot them easily

In [138]:
not_nan = ~np.isnan(Z[0])
SA = xr.DataArray(
    SA[:, not_nan],
    dims=["TIME", "DEPTH"],
    coords={"TIME": single_profile.TIME.values, "DEPTH": Z[0, not_nan]},
)
CT = xr.DataArray(
    CT[:, not_nan],
    dims=["TIME", "DEPTH"],
    coords={"TIME": single_profile.TIME.values, "DEPTH": Z[0, not_nan]},
)
RHO = xr.DataArray(
    RHO[:, not_nan],
    dims=["TIME", "DEPTH"],
    coords={"TIME": single_profile.TIME.values, "DEPTH": Z[0, not_nan]},
)

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(10, 10))
RHO.plot(
    ax=axs[0], x="TIME", cmap=cmo.cm.dense, cbar_kwargs={"label": "Density (kg/m^3)"}
)
cont = RHO.plot.contour(ax=axs[0], x="TIME", colors="k", levels=10, linewidths=0.5)
axs[0].clabel(cont, inline=True, fontsize=8, fmt="%1.2f")

SA.plot(
    ax=axs[1],
    x="TIME",
    cmap=cmo.cm.haline,
    cbar_kwargs={"label": "Absolute Salinity (g/kg)"},
)
cont = SA.plot.contour(ax=axs[1], x="TIME", colors="k", levels=10, linewidths=0.5)
axs[1].clabel(cont, inline=True, fontsize=8, fmt="%1.2f")

CT.plot(
    ax=axs[2],
    x="TIME",
    cmap=cmo.cm.thermal,
    cbar_kwargs={"label": "Conservative Temperature (C)"},
)
cont = CT.plot.contour(ax=axs[2], x="TIME", colors="k", levels=10, linewidths=0.5)
axs[2].clabel(cont, inline=True, fontsize=8, fmt="%1.2f")

for _ax in axs:
    _ax.set_ylim(-1000, 0)
    _ax.set_xlim(single_profile.TIME.min(), single_profile.TIME.max())
    _ax.grid(ls="dotted", alpha=0.5)